In [2]:
import workflow.scripts.RESources as RES
import pandas as pd
import geopandas as gpd

In [3]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis
store=f"data/store/resources_BC.h5"
datahandler=DataHandler(store)

In [4]:
cells=datahandler.from_store('cells')

In [ ]:
cells['LCOE_wind']

# Set Required Args to Activate Modules

In [ ]:
import workflow.scripts.RESources as RES
import pandas as pd
import geopandas as gpd

# Iterate over provinces for both solar and wind resources
resource_types = ['wind','solar'] 
provinces=['BC']  #'AB','SK','ON','NS'
for province_code in provinces:
    for resource_type in resource_types:
        required_args = {
            "config_file_path": 'config/config.yaml',
            "province_short_code": province_code,
            "resource_type": resource_type
        }
        
        # Create an instance of Resources and execute the module
        RES_module = RES.RESources_builder(**required_args)
        RES_module.build(use_pypsa_buses=False)